# Introduction to Supercells

Within CASM, a configuration is used to represent a single crystal state.

A configuration is itself defined in terms of:
- The primitive crystal structure and allowed degrees of freedom (DoF) (the "prim").
- A supercell, which specifies the translational periodicity of the configuration. 
- DoF values, including:
  - Site DoF "within" the supercell (i.e. the translationally unique part of the crystal state)
    - Discrete occupation DoF (atom type)
    - Continuous site DoF (displacment or magnetic spin) 
  - Continuous global DoF (strain)


Supercells may be generated automatically when enumerating configurations, but it is also common to enumerate supercells independently and then choose which supercells to enumerate configurations in.

## Supercell lattices

Supercell lattices are multiples of the prim lattice: 

$$ L^{scel} = L^{prim} * T, $$

where:

- $T$ is a 3x3 integer transformation matrix
- $L^{scel}$ and $L^{prim}$ are the supercell and prim lattices, as column vector matrices 


## Equivalence of supercell lattices

Two supercell lattices, $L^{scel}$ and ${L^{scel}}^{\prime}$ are symmetrically equivalent if:

$$ {L^{scel}}^{\prime} = A*L^{scel}*T, $$

where:

- $A$ is a crystal point group operation matrix
- $T$ is an integer transformation matrix

Of all the symmetrically equivalent supercell lattices, CASM deterministically chooses one to be the "canonical" supercell lattice, based on an algorithm that favors symmetric, positive, diagonal $T$.

## Supercell naming convention

CASM gives supercells a unique name of the form `SCELV_A_B_C_D_E_F` based on:
- The supercell volume, V, which is the determinant of the supercell transformation matrix, $T$
- The six non-zero elements (A-F) of the hermite normal form of $T$. The hermite normal form 


# Supercell Enumeration

The `ScelEnum` method of `enum` enables enumerating supercells. It includes options to control enumerating supercells within a range of volumes, as multiples of a non-primitive unit cell, and restricted to particular directions (i.e. 1d or 2d supercells).

In the following we give examples of supercell enumeration. A complete list of `ScelEnum` method parameters can be obtained with:

    ccasm enum --desc ScelEnum
    
The examples will all use this prim structure for the system of HCP Zr with O or vacancy occupation of the octahedral interstitial sites:

In [ ]:
prim_str='
{
  "basis" : [
    {
      "coordinate" : [ 0.0000000, 0.0000000, 0.0000000 ],
      "occupants" : [ "Zr" ]
    },
    {
      "coordinate" : [ 0.6666666667, 0.3333333333, 0.5000000 ],
      "occupants" : [ "Zr" ]
    },
    {
      "coordinate" : [ 0.3333333333, 0.6666666667, 0.2500000 ],
      "occupants" : [ "Va", "O" ]
    },
    {
      "coordinate" : [ 0.3333333333, 0.6666666667, 0.7500000 ],
      "occupants" : [ "Va", "O" ]
    }
  ],
  "coordinate_mode" : "Fractional",
  "description" : "hcp Zr with octahedral interstitial O ",
  "lattice_vectors" : [
    [ 3.23398686, 0.00000000, 0.00000000 ],
    [ -1.61699343, 2.80071477, 0.00000000 ],
    [ -0.00000000, 0.00000000, 5.16867834 ]
  ],
  "title" : "ZrO"
}'

In [ ]:
start=$(pwd)
mkdir -p $start/enum/ZrO && cd $start/enum/ZrO

echo "$prim_str" > prim.json
ccasm init

## Enumerate supercells up to a maximum volume

Enumerate all supercells of volume 4 or less.

In [ ]:
ccasm enum -m ScelEnum -i '{"max": 4}'

## Enumerate supercells within a range of volumes

Enumerate all supercells of volume 6 or 7:

In [ ]:
ccasm enum -m ScelEnum -i '{"min": 6, "max": 7}'

## Enumerate supercells of a non-primitive unit cell

Using the 2x2x2 supercell as the base unit cell, enumerate up to volume 4 times its volume:

In [ ]:
ccasm enum -m ScelEnum -i '{
  "max": 4, 
  "unit_cell": [
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2]
  ]
}'

## Restrict supercell enumeration to particular directions

<i>Note: The "dirs" values "a", "b", "c" indicate the first, second, and third lattice vector after applying "unit_cell".</i>

Using the 2x2x2 supercell as the basis unit cell, enumerate 2d supercells in the HCP basal plane. 

In [ ]:
ccasm enum -m ScelEnum -i '{
  "max": 9, 
  "unit_cell": [
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2]
  ],
  "dirs": "ab"
}'

# Selecting and Querying Enumerated Supercells

Use the `-t scel` option to indicate `query` and `select` operations on supercells. CASM stores a default list of selected supercells called the `MASTER` selection within the ".casm" directory. Other standard selections CASM defines that can be used with the `-c` option are: 

- `All`: all supercells in the database included and selected
- `NONE`: all supercells in the database included but none selected
- `EMPTY`: no supercells included

_Note: `CALCULATED` is not allowed for supercells_

Custom user lists can be generated with `-o <filename>` and used with `-c <filename>`.

## Select all enumerated supercells

In [ ]:
ccasm select -t scel --set-on

## Query properties of selected supercells

- By default, the MASTER selection is queried
- A list of properties that can be queried can be printed with:

      ccasm query --help properties
 
In this example, the supercell volume as a multiple of the prim is queried (`scel_size`) and the results are written to a file name  `all_supercells_and_size`. Since the file is large, we only print the  beginning of the file here for an example.

In [ ]:
ccasm query -t scel -k scel_size > all_supercells_and_size
head all_supercells_and_size

## Combine selections and queries with operators

- Queried properties can be combined through boolean and comparison operations.
- A list of all operators can be printed with:

      ccasm query --help operators

This example selects all volume 4 supercells and saves them to a custom selection file named `volume_4_scel_list.txt`.

In [ ]:
ccasm select -t scel -c ALL --set 'eq(scel_size,4)' -o volume_4_scel_list.txt

## Using a custom selection, writing a JSON output file

This example queries some properties of just the volume 4 supercells and saves the output as a JSON file named `volume_4_props.json`. 

JSON output is obtained either by giving the output file a `.json` extension, or using the `--json,-j` option.

In [ ]:
ccasm query -t scel -c volume_4_scel_list.txt -k multiplicity pointgroup_name lattice_params -o volume_4_props.json
head -n 30 volume_4_props.json

# Getting Supercell `info`

Sometimes it is useful to get information about a supercell outside of the context of a CASM project.

The `info` command allows querying properties of supercells without first creating a CASM project and enumerating supercells, although it can also be used within the context of a project.

The `SupercellInfo` method allows for getting information about the supercell lattice vectors, the ordering of the representation of DoF values in a configuration, permutation representations, and linear site ordering. 

To see a list of all supercell properties that may be obtained from the `SupercellInfo` method, and a description of their representation, use:

     ccasm info --desc SupercellInfo
 

## Query supercell site coordinates

This example queries the coordinates of the sites in a supercell, in Cartesian, fractional, and integral site coordinates.

Fractional coordinates in this context are defined as multiples of the supercell lattice vectors.

Integral site coordinates specify the position of a site using integers `[b, i, j, k]`, representing (`b`) the site's sublattice as an index into the in the prim basis, and (`i, j, k`) the unit cell it is in using multiples of the prim lattice vectors.

In [ ]:
supercell_info_str="{
  \"prim\":$prim_str,
  \"transformation_matrix_to_super\": [[2, 0, 0], [0, 2, 0], [0, 0, 2]],
  \"properties\": [\"cart_coordinate\", \"frac_coordinate\", \"integral_site_coordinates\"] 
}"
ccasm info -m SupercellInfo -i "$supercell_info_str"

Clean up tutorial data:

In [ ]:
cd $start && rm -r $start/enum